In [3]:
import pandas as pd
from openai import OpenAI
from openpyxl import load_workbook
from openpyxl.drawing.image import Image
import requests
import os
import json
import time

# 配置 OpenAI 客户端
client = OpenAI(base_url="https://chatapi.littlewheat.com/v1", api_key="sk-w33LIKFEXz7UHKg2R5tVvboSNAU76iCZsvyOyOjPtLLPp89S")  # 替换为你的 API Key

# 读取 Excel 文件
file_path = r"C:\Users\86155\Downloads\新浪网_sb.xlsx"  # 填写你的文件路径
df = pd.read_excel(file_path)
empty_rows=df[df.isnull().all(axis=1)].index
first_empty_row=empty_rows[0] if not empty_rows.empty else len(df)
df=df.iloc[:20]# 找到第一个全为空的行，截断后续行
#iloc冒号之前的数字n代表从第n+1条新闻开始处理，为空时等同于n=0;冒号之后的数字代表处理到第几条，填first_empty_row则全处理
# 添加问题一到问题七的列

df['问题一'] = ''

# 获取需要插入数据的列（从第九列到第十五列，图片插入第十六列）
questions = """
请回答以下问题：
-问题一：将内容概括或扩充至200字到250字之间，要求内容与加勒比地区有关，并且只有一个段落不分行
请严格返回 JSON 格式的结果，返回格式：{"问题一": "xxx"}
"""

# 调用 GPT API 获取问题回答
def call_gpt_api(message, max_token, retries):
    for _ in range(retries):
        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=message,
                max_tokens=max_token,
                timeout=30
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"API调用失败: {e}, 重试中...")
            time.sleep(1)  # 未响应重试
    return ""

# 获取问题答案
def get_answers(text):
    kinds = [{"role": "system", "content": "你是一个熟知加勒比地区的资深新闻分析师。"},
             {"role": "user", "content": f"通读新闻:{text}，{questions}"}]
    response_content = call_gpt_api(kinds, 500, 3)
    try:
        answers = json.loads(response_content)
        return answers.get("问题一", "")
    except json.JSONDecodeError:
        print("问题回答过程出现问题")
        return "未知"

def fix_url(url):
    #"""确保 URL 以 http:// 或 https:// 开头"""
    if not url.startswith(("http://", "https://")):
        return "https:" + url  # 默认加 https
    return url

def convert_webp_to_jpg(input_path, output_path):
    with PILImage.open(input_path) as img:
        img.convert("RGB").save(output_path, "JPEG")
        
# 定义图片保存的文件夹
output_folder = '图库'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 加载 Excel 文件，准备插入图片和答案
wb = load_workbook(file_path)
ws = wb.active  # 默认活动工作表

# 图片插入的列索引（16列，即Excel中的第16列）
image_column = 16

# 遍历每一行新闻，进行问题回答并插入图片
for i, row in df.iterrows():
    print(f"正在处理第{i + 1}/{len(df)}条新闻...")

    # 获取新闻的正文和标题
    text = row['正文']

    # 获取问题一到问题七的答案
    answers = get_answers(text)  # 假设 get_answers 是一个自定义函数

    # 获取图片链接
    image_url = row['图片链接']
    if not image_url:
        print(f"第{i + 1}条新闻没有图片链接，跳过该条目。")
        continue  # 如果没有图片链接，跳过当前条目

    # 修正 URL，防止 MissingSchema 错误
    image_url = fix_url(image_url)

    # 设置图片保存路径
    img_name = f"image_{i + 1}.jpg"
    img_path = os.path.join(output_folder, img_name)

    try:
        # 请求图片内容
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'
        }
        img_response = requests.get(image_url, headers=headers, timeout=10)

        # 检查图片是否成功下载
        if img_response.status_code == 200:
            with open(img_path, 'wb') as f:
                f.write(img_response.content)
            print(f"下载图片成功: {img_name}")

            # 如果图片是 .webp 格式，先转换为 .jpg 格式
            if img_name.endswith(".webp"):
                jpg_img_name = img_name.replace(".webp", ".jpg")
                jpg_img_path = os.path.join(output_folder, jpg_img_name)
                convert_webp_to_jpg(img_path, jpg_img_path)
                img_path = jpg_img_path  # 更新图片路径为转换后的 jpg 路径
                print(f"图片已转换为 JPG 格式: {jpg_img_name}")

            # 插入问题一到问题七的答案到Excel
            ws.cell(row=i + 2, column=10, value=answers[0])  # 问题
            

            # 插入图片到Excel中的第十六列
            img = Image(img_path)
            img.width = 200  # 设置图片宽度
            img.height = 150  # 设置图片高度
            ws.add_image(img, f"P{i+2}")  # P列即Excel中的第十六列

            i += 1  # 更新 Excel 行号
        else:
            print(f"图片下载失败: {img_name}, 状态码: {img_response.status_code}")
    except Exception as e:
        print(f"处理第{i + 1}条新闻时发生错误: {e}")

# 保存修改后的Excel文件
new_file_path = r"C:\Users\86155\Desktop\学习\3.xlsx"
wb.save(new_file_path)
print(f"所有新闻处理完毕，已保存至：{new_file_path}")

正在处理第1/20条新闻...
下载图片成功: image_1.jpg
正在处理第2/20条新闻...
下载图片成功: image_2.jpg
正在处理第3/20条新闻...
下载图片成功: image_3.jpg
正在处理第4/20条新闻...
图片下载失败: image_4.jpg, 状态码: 403
正在处理第5/20条新闻...
图片下载失败: image_5.jpg, 状态码: 403
正在处理第6/20条新闻...
下载图片成功: image_6.jpg
正在处理第7/20条新闻...
图片下载失败: image_7.jpg, 状态码: 403
正在处理第8/20条新闻...
下载图片成功: image_8.jpg
正在处理第9/20条新闻...
图片下载失败: image_9.jpg, 状态码: 403
正在处理第10/20条新闻...
下载图片成功: image_10.jpg
正在处理第11/20条新闻...
下载图片成功: image_11.jpg
正在处理第12/20条新闻...
图片下载失败: image_12.jpg, 状态码: 403
正在处理第13/20条新闻...
下载图片成功: image_13.jpg
正在处理第14/20条新闻...
下载图片成功: image_14.jpg
正在处理第15/20条新闻...
下载图片成功: image_15.jpg
正在处理第16/20条新闻...
图片下载失败: image_16.jpg, 状态码: 403
正在处理第17/20条新闻...
下载图片成功: image_17.jpg
正在处理第18/20条新闻...
下载图片成功: image_18.jpg
正在处理第19/20条新闻...
下载图片成功: image_19.jpg
正在处理第20/20条新闻...
图片下载失败: image_20.jpg, 状态码: 403
所有新闻处理完毕，已保存至：C:\Users\86155\Desktop\学习\3.xlsx
